In [1]:
import pandas as pd
import numpy as np


import nltk

import warnings

In [2]:
df = pd.read_csv(r"C:\Users\mitpa\Downloads\reviews.csv",encoding='unicode_escape')
print(df.shape)

(67986, 8)


In [3]:
#sorting
df = df.head(500)
print(df.shape)
df.head()


(500, 8)


,asin,name,rating,date,verified,title,body,helpfulVotes
0,B0000SX2UC,Janet,3,"October 11, 2005",False,"Def not best, but not worst",I had the Samsung A600 for awhile which is abs...,1.0
1,B0000SX2UC,Luke Wyatt,1,"January 7, 2004",False,Text Messaging Doesn't Work,Due to a software issue between Nokia and Spri...,17.0
2,B0000SX2UC,Brooke,5,"December 30, 2003",False,Love This Phone,"This is a great, reliable phone. I also purcha...",5.0
3,B0000SX2UC,amy m. teague,3,"March 18, 2004",False,"Love the Phone, BUT...!","I love the phone and all, because I really did...",1.0
4,B0000SX2UC,tristazbimmer,4,"August 28, 2005",False,"Great phone service and options, lousy case!",The phone has been great for every purpose it ...,1.0


In [4]:
example = df['body'][4]
print(example)
nltk.word_tokenize(example)

The phone has been great for every purpose it offers, except the day i bought it-i couldnt get the case off. You can take the case off to put your own pictures in the jaket which is super cool, but when i took it back to the store-an employee said they were meant to be hard to get off. Well, he could barely even get it off- he came close to snapping the casing in half. I was never able to get it off after that which isnt THAT big of a deal, but it got dirty under the clear casing and it looks really dirty now. MAKE SURE YOU CAN GET YOUR CASE ON AND OFF! Oh and you have to charge it ALL off the time.


['The',
 'phone',
 'has',
 'been',
 'great',
 'for',
 'every',
 'purpose',
 'it',
 'offers',
 ',',
 'except',
 'the',
 'day',
 'i',
 'bought',
 'it-i',
 'couldnt',
 'get',
 'the',
 'case',
 'off',
 '.',
 'You',
 'can',
 'take',
 'the',
 'case',
 'off',
 'to',
 'put',
 'your',
 'own',
 'pictures',
 'in',
 'the',
 'jaket',
 'which',
 'is',
 'super',
 'cool',
 ',',
 'but',
 'when',
 'i',
 'took',
 'it',
 'back',
 'to',
 'the',
 'store-an',
 'employee',
 'said',
 'they',
 'were',
 'meant',
 'to',
 'be',
 'hard',
 'to',
 'get',
 'off',
 '.',
 'Well',
 ',',
 'he',
 'could',
 'barely',
 'even',
 'get',
 'it',
 'off-',
 'he',
 'came',
 'close',
 'to',
 'snapping',
 'the',
 'casing',
 'in',
 'half',
 '.',
 'I',
 'was',
 'never',
 'able',
 'to',
 'get',
 'it',
 'off',
 'after',
 'that',
 'which',
 'isnt',
 'THAT',
 'big',
 'of',
 'a',
 'deal',
 ',',
 'but',
 'it',
 'got',
 'dirty',
 'under',
 'the',
 'clear',
 'casing',
 'and',
 'it',
 'looks',
 'really',
 'dirty',
 'now',
 '.',
 'MAKE',
 'SURE'

In [5]:
#vader model start

from nltk.sentiment import SentimentIntensityAnalyzer
from tqdm.notebook import tqdm


sia = SentimentIntensityAnalyzer()
sia.polarity_scores('I love this movie!')

# from nltk.sentiment.vader import SentimentIntensityAnalyzer

# sid = SentimentIntensityAnalyzer()
# scores = sid.polarity_scores("I love this movie!")
# print(scores)

#opposite

sia.polarity_scores('This is a worst movie ever!')

{'neg': 0.523, 'neu': 0.477, 'pos': 0.0, 'compound': -0.6588}

In [6]:
sia.polarity_scores(example)

{'neg': 0.066, 'neu': 0.814, 'pos': 0.121, 'compound': 0.7745}

In [7]:
# Run the polarity score on the entire dataset
res = {}
for i, row in tqdm(df.iterrows(), total=len(df)):
    text = row['body']
    myid = i  # Use the index as the unique identifier
    res[myid] = sia.polarity_scores(text)

  0%|          | 0/500 [00:00<?, ?it/s]

In [8]:
df['Id'] = range(len(df))  # Adding a unique identifier column
res = {}
for i, row in tqdm(df.iterrows(), total=len(df)):
    text = row['body']
    myid = row['Id']  # Now 'Id' exists in the DataFrame
    res[myid] = sia.polarity_scores(text)


  0%|          | 0/500 [00:00<?, ?it/s]

In [9]:
# res

In [10]:
vaders = pd.DataFrame(res).T
vaders = vaders.reset_index().rename(columns={'index': 'Id'})
vaders = vaders.merge(df, how='left')
vaders.head()

,Id,neg,neu,pos,compound,asin,name,rating,date,verified,title,body,helpfulVotes
0,0,0.080,0.816,0.105,0.8629,B0000SX2UC,Janet,3,"October 11, 2005",False,"Def not best, but not worst",I had the Samsung A600 for awhile which is abs...,1.0
1,1,0.020,0.876,0.104,0.8860,B0000SX2UC,Luke Wyatt,1,"January 7, 2004",False,Text Messaging Doesn't Work,Due to a software issue between Nokia and Spri...,17.0
2,2,0.051,0.846,0.103,0.7992,B0000SX2UC,Brooke,5,"December 30, 2003",False,Love This Phone,"This is a great, reliable phone. I also purcha...",5.0
3,3,0.000,0.844,0.156,0.9592,B0000SX2UC,amy m. teague,3,"March 18, 2004",False,"Love the Phone, BUT...!","I love the phone and all, because I really did...",1.0
4,4,0.066,0.814,0.121,0.7745,B0000SX2UC,tristazbimmer,4,"August 28, 2005",False,"Great phone service and options, lousy case!",The phone has been great for every purpose it ...,1.0


In [11]:
def sentiment_vader(body):
    overall_polarity = sia.polarity_scores(body)
    if overall_polarity['compound'] > 0.5:
        return "Positive"
    elif overall_polarity['compound'] <-0.5:
        return "Negative" 
    else:
        return "Neutral"

In [12]:
vresults_df = pd.DataFrame(vaders)
vresults_df['Sentiment_vader'] = vresults_df['body'].apply(lambda x: sentiment_vader(x))
vresults_df.head(20)

,Id,neg,neu,pos,compound,asin,name,rating,date,verified,title,body,helpfulVotes,Sentiment_vader
0,0,0.080,0.816,0.105,0.8629,B0000SX2UC,Janet,3,"October 11, 2005",False,"Def not best, but not worst",I had the Samsung A600 for awhile which is abs...,1.0,Positive
1,1,0.020,0.876,0.104,0.8860,B0000SX2UC,Luke Wyatt,1,"January 7, 2004",False,Text Messaging Doesn't Work,Due to a software issue between Nokia and Spri...,17.0,Positive
2,2,0.051,0.846,0.103,0.7992,B0000SX2UC,Brooke,5,"December 30, 2003",False,Love This Phone,"This is a great, reliable phone. I also purcha...",5.0,Positive
3,3,0.000,0.844,0.156,0.9592,B0000SX2UC,amy m. teague,3,"March 18, 2004",False,"Love the Phone, BUT...!","I love the phone and all, because I really did...",1.0,Positive
4,4,0.066,0.814,0.121,0.7745,B0000SX2UC,tristazbimmer,4,"August 28, 2005",False,"Great phone service and options, lousy case!",The phone has been great for every purpose it ...,1.0,Positive
5,5,0.068,0.782,0.150,0.7066,B0000SX2UC,J. White,4,"September 25, 2005",False,Worked great for me,"Hello, I have this phone and used it until I d...",NaN,Positive
6,6,0.037,0.777,0.186,0.9805,B0000SX2UC,the cell phone store owner,5,"April 16, 2004",False,Wanna cool Nokia? You have it here!,Cool. Cheap. Color: 3 words that describe the ...,2.0,Positive
7,7,0.000,0.912,0.088,0.6705,B0000SX2UC,Matt,4,"April 3, 2004",False,Problem with 3588i universal headset,"The 3599i is overall a nice phone, except that...",2.0,Positive
8,8,0.038,0.727,0.234,0.8953,B0000SX2UC,Charles Cook,5,"November 24, 2003",False,cool phone!!!!!!!!,"I've never owned a Nokia phone before, so this...",7.0,Positive
9,9,0.030,0.870,0.100,0.6486,B0000SX2UC,Amazon Customer,3,"February 2, 2004",False,Pissed off-a little bit,ok well im in school and i need the text messa...,3.0,Positive


In [13]:
vader_df = pd.DataFrame(vresults_df)
vader_df

,Id,neg,neu,pos,compound,asin,name,rating,date,verified,title,body,helpfulVotes,Sentiment_vader
0,0,0.080,0.816,0.105,0.8629,B0000SX2UC,Janet,3,"October 11, 2005",False,"Def not best, but not worst",I had the Samsung A600 for awhile which is abs...,1.0,Positive
1,1,0.020,0.876,0.104,0.8860,B0000SX2UC,Luke Wyatt,1,"January 7, 2004",False,Text Messaging Doesn't Work,Due to a software issue between Nokia and Spri...,17.0,Positive
2,2,0.051,0.846,0.103,0.7992,B0000SX2UC,Brooke,5,"December 30, 2003",False,Love This Phone,"This is a great, reliable phone. I also purcha...",5.0,Positive
3,3,0.000,0.844,0.156,0.9592,B0000SX2UC,amy m. teague,3,"March 18, 2004",False,"Love the Phone, BUT...!","I love the phone and all, because I really did...",1.0,Positive
4,4,0.066,0.814,0.121,0.7745,B0000SX2UC,tristazbimmer,4,"August 28, 2005",False,"Great phone service and options, lousy case!",The phone has been great for every purpose it ...,1.0,Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,495,0.000,0.313,0.687,0.6588,B002WTC1NG,Ryan M.,4,"December 16, 2014",True,Four Stars,Great flip phone!,NaN,Positive
496,496,0.000,1.000,0.000,0.0000,B002WTC1NG,Ronald G. Dolter,5,"April 9, 2015",True,Five Stars,It arrived within the time quoted and is funct...,NaN,Neutral
497,497,0.000,0.804,0.196,0.2960,B002WTC1NG,Thomas B. Reece,3,"May 14, 2015",True,Three Stars,"Phone worked ok, however the charger NEVER wor...",NaN,Neutral
498,498,0.000,0.758,0.242,0.4927,B002WTC1NG,sasha ford,5,"March 26, 2015",True,Five Stars,Very pleased with the speedy arrival and quali...,NaN,Neutral


In [14]:
def remove_unnecessary_columns(vader_df, columns_to_remove):
    for column in columns_to_remove:
        if column in vader_df.columns:
            vader_df.drop(column, axis=1, inplace=True)
        else:
            break

    return vader_df

In [15]:
vader_df =vader_df[['Id','neg','pos','neu','compound','asin','name','date','verified','title','body','helpfulVotes','Sentiment_vader']]

columns_to_remove = ['verified','helpfulVotes']

vader_df = remove_unnecessary_columns(vader_df, columns_to_remove)
vader_df.head(500)

C:\Users\mitpa\AppData\Local\Temp\ipykernel_18892\4085925251.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vader_df.drop(column, axis=1, inplace=True)
C:\Users\mitpa\AppData\Local\Temp\ipykernel_18892\4085925251.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vader_df.drop(column, axis=1, inplace=True)


,Id,neg,pos,neu,compound,asin,name,date,title,body,Sentiment_vader
0,0,0.080,0.105,0.816,0.8629,B0000SX2UC,Janet,"October 11, 2005","Def not best, but not worst",I had the Samsung A600 for awhile which is abs...,Positive
1,1,0.020,0.104,0.876,0.8860,B0000SX2UC,Luke Wyatt,"January 7, 2004",Text Messaging Doesn't Work,Due to a software issue between Nokia and Spri...,Positive
2,2,0.051,0.103,0.846,0.7992,B0000SX2UC,Brooke,"December 30, 2003",Love This Phone,"This is a great, reliable phone. I also purcha...",Positive
3,3,0.000,0.156,0.844,0.9592,B0000SX2UC,amy m. teague,"March 18, 2004","Love the Phone, BUT...!","I love the phone and all, because I really did...",Positive
4,4,0.066,0.121,0.814,0.7745,B0000SX2UC,tristazbimmer,"August 28, 2005","Great phone service and options, lousy case!",The phone has been great for every purpose it ...,Positive
...,...,...,...,...,...,...,...,...,...,...,...
495,495,0.000,0.687,0.313,0.6588,B002WTC1NG,Ryan M.,"December 16, 2014",Four Stars,Great flip phone!,Positive
496,496,0.000,0.000,1.000,0.0000,B002WTC1NG,Ronald G. Dolter,"April 9, 2015",Five Stars,It arrived within the time quoted and is funct...,Neutral
497,497,0.000,0.196,0.804,0.2960,B002WTC1NG,Thomas B. Reece,"May 14, 2015",Three Stars,"Phone worked ok, however the charger NEVER wor...",Neutral
498,498,0.000,0.242,0.758,0.4927,B002WTC1NG,sasha ford,"March 26, 2015",Five Stars,Very pleased with the speedy arrival and quali...,Neutral


In [16]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax

In [17]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

In [18]:
example = df['body'][50]
print(example)

This is not a "boost" phone and will not work as a GPS with Boost. Had to send it back. A huge pain for something that is $24. This is a rip-off.


In [19]:
# VADER results on example
print(example)
sia.polarity_scores(example)

This is not a "boost" phone and will not work as a GPS with Boost. Had to send it back. A huge pain for something that is $24. This is a rip-off.


{'neg': 0.099, 'neu': 0.751, 'pos': 0.15, 'compound': 0.1779}

In [20]:
example = df['body'][50]
print(example)


This is not a "boost" phone and will not work as a GPS with Boost. Had to send it back. A huge pain for something that is $24. This is a rip-off.


In [21]:
# Run for Roberta Model
encoded_text = tokenizer(example, return_tensors='pt')
output = model(**encoded_text)
scores = output[0][0].detach().numpy()
scores = softmax(scores)
scores_dict = {
    'roberta_neg' : scores[0],
    'roberta_neu' : scores[1],
    'roberta_pos' : scores[2]
}
print(scores_dict)

{'roberta_neg': 0.92612135, 'roberta_neu': 0.06435084, 'roberta_pos': 0.0095278425}


In [22]:
def polarity_scores_roberta(example):
    encoded_text = tokenizer(example, return_tensors='pt')
    output = model(**encoded_text)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    scores_dict = {
        'roberta_neg' : scores[0],
        'roberta_neu' : scores[1],
        'roberta_pos' : scores[2]
    }
    return scores_dict

In [23]:
res1 = {}
for i, row in tqdm(df.iterrows(), total=len(df)):
    try:
        body = row['body']
        myid = row['Id']
        vader_result = sia.polarity_scores(body)
        vader_result_rename = {}
        for key, value in vader_result.items():
            vader_result_rename[f"vader_{key}"] = value
        roberta_result = polarity_scores_roberta(body)
        both = {**vader_result_rename, **roberta_result}
        res1[myid] = both
    except RuntimeError:
        print(f'Broke for id {myid}')

  0%|          | 0/500 [00:00<?, ?it/s]

Broke for id 19
Broke for id 40
Broke for id 82
Broke for id 84
Broke for id 100
Broke for id 145


In [24]:
# res1

In [25]:
results_df = pd.DataFrame(res1).T

results_df = results_df.reset_index().rename(columns={'index': 'Id'})
results_df = results_df.merge(df, how='left')


In [26]:
results_df.head()


,Id,vader_neg,vader_neu,vader_pos,vader_compound,roberta_neg,roberta_neu,roberta_pos,asin,name,rating,date,verified,title,body,helpfulVotes
0,0,0.080,0.816,0.105,0.8629,0.387844,0.342091,0.270064,B0000SX2UC,Janet,3,"October 11, 2005",False,"Def not best, but not worst",I had the Samsung A600 for awhile which is abs...,1.0
1,1,0.020,0.876,0.104,0.8860,0.459493,0.425768,0.114739,B0000SX2UC,Luke Wyatt,1,"January 7, 2004",False,Text Messaging Doesn't Work,Due to a software issue between Nokia and Spri...,17.0
2,2,0.051,0.846,0.103,0.7992,0.134225,0.299039,0.566736,B0000SX2UC,Brooke,5,"December 30, 2003",False,Love This Phone,"This is a great, reliable phone. I also purcha...",5.0
3,3,0.000,0.844,0.156,0.9592,0.009765,0.021803,0.968432,B0000SX2UC,amy m. teague,3,"March 18, 2004",False,"Love the Phone, BUT...!","I love the phone and all, because I really did...",1.0
4,4,0.066,0.814,0.121,0.7745,0.236579,0.328686,0.434734,B0000SX2UC,tristazbimmer,4,"August 28, 2005",False,"Great phone service and options, lousy case!",The phone has been great for every purpose it ...,1.0


In [27]:
dresults_df = pd.DataFrame(results_df)
dresults_df.head()

,Id,vader_neg,vader_neu,vader_pos,vader_compound,roberta_neg,roberta_neu,roberta_pos,asin,name,rating,date,verified,title,body,helpfulVotes
0,0,0.080,0.816,0.105,0.8629,0.387844,0.342091,0.270064,B0000SX2UC,Janet,3,"October 11, 2005",False,"Def not best, but not worst",I had the Samsung A600 for awhile which is abs...,1.0
1,1,0.020,0.876,0.104,0.8860,0.459493,0.425768,0.114739,B0000SX2UC,Luke Wyatt,1,"January 7, 2004",False,Text Messaging Doesn't Work,Due to a software issue between Nokia and Spri...,17.0
2,2,0.051,0.846,0.103,0.7992,0.134225,0.299039,0.566736,B0000SX2UC,Brooke,5,"December 30, 2003",False,Love This Phone,"This is a great, reliable phone. I also purcha...",5.0
3,3,0.000,0.844,0.156,0.9592,0.009765,0.021803,0.968432,B0000SX2UC,amy m. teague,3,"March 18, 2004",False,"Love the Phone, BUT...!","I love the phone and all, because I really did...",1.0
4,4,0.066,0.814,0.121,0.7745,0.236579,0.328686,0.434734,B0000SX2UC,tristazbimmer,4,"August 28, 2005",False,"Great phone service and options, lousy case!",The phone has been great for every purpose it ...,1.0


In [28]:
def sentiment_vader(body):
    overall_polarity = sia.polarity_scores(body)
    if overall_polarity['compound'] >= 0.05:
        return "Positive"
    elif overall_polarity['compound'] <= -0.05:
        return "Negative" 
    else:
        return "Neutral"


In [29]:
dresults_df['Sentiment_vader'] = results_df['body'].apply(lambda x: sentiment_vader(x))
dresults_df.head(500)

,Id,vader_neg,vader_neu,vader_pos,vader_compound,roberta_neg,roberta_neu,roberta_pos,asin,name,rating,date,verified,title,body,helpfulVotes,Sentiment_vader
0,0,0.080,0.816,0.105,0.8629,0.387844,0.342091,0.270064,B0000SX2UC,Janet,3,"October 11, 2005",False,"Def not best, but not worst",I had the Samsung A600 for awhile which is abs...,1.0,Positive
1,1,0.020,0.876,0.104,0.8860,0.459493,0.425768,0.114739,B0000SX2UC,Luke Wyatt,1,"January 7, 2004",False,Text Messaging Doesn't Work,Due to a software issue between Nokia and Spri...,17.0,Positive
2,2,0.051,0.846,0.103,0.7992,0.134225,0.299039,0.566736,B0000SX2UC,Brooke,5,"December 30, 2003",False,Love This Phone,"This is a great, reliable phone. I also purcha...",5.0,Positive
3,3,0.000,0.844,0.156,0.9592,0.009765,0.021803,0.968432,B0000SX2UC,amy m. teague,3,"March 18, 2004",False,"Love the Phone, BUT...!","I love the phone and all, because I really did...",1.0,Positive
4,4,0.066,0.814,0.121,0.7745,0.236579,0.328686,0.434734,B0000SX2UC,tristazbimmer,4,"August 28, 2005",False,"Great phone service and options, lousy case!",The phone has been great for every purpose it ...,1.0,Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,495,0.000,0.313,0.687,0.6588,0.001679,0.016688,0.981633,B002WTC1NG,Ryan M.,4,"December 16, 2014",True,Four Stars,Great flip phone!,NaN,Positive
490,496,0.000,1.000,0.000,0.0000,0.004309,0.207647,0.788044,B002WTC1NG,Ronald G. Dolter,5,"April 9, 2015",True,Five Stars,It arrived within the time quoted and is funct...,NaN,Neutral
491,497,0.000,0.804,0.196,0.2960,0.470821,0.396961,0.132217,B002WTC1NG,Thomas B. Reece,3,"May 14, 2015",True,Three Stars,"Phone worked ok, however the charger NEVER wor...",NaN,Positive
492,498,0.000,0.758,0.242,0.4927,0.001344,0.013701,0.984955,B002WTC1NG,sasha ford,5,"March 26, 2015",True,Five Stars,Very pleased with the speedy arrival and quali...,NaN,Positive


In [30]:
vader_df = pd.DataFrame(dresults_df)


In [31]:
def remove_unnecessary_columns(vader_df, columns_to_remove):
    for column in columns_to_remove:
        if column in vader_df.columns:
            vader_df.drop(column, axis=1, inplace=True)
        else:
            break

    return vader_df

In [32]:
vader_df =vader_df[['Id','vader_neg','vader_pos','vader_neu','vader_compound','roberta_neg','roberta_neu','roberta_pos','asin','name','date','verified','title','body','helpfulVotes','Sentiment_vader']]

columns_to_remove = ['roberta_neg','vader_neg','vader_pos','vader_neu','vader_compound','roberta_neu','roberta_pos','verified','helpfulVotes']

vader_df = remove_unnecessary_columns(vader_df, columns_to_remove)
vader_df.head(500)

,Id,asin,name,date,title,body,Sentiment_vader
0,0,B0000SX2UC,Janet,"October 11, 2005","Def not best, but not worst",I had the Samsung A600 for awhile which is abs...,Positive
1,1,B0000SX2UC,Luke Wyatt,"January 7, 2004",Text Messaging Doesn't Work,Due to a software issue between Nokia and Spri...,Positive
2,2,B0000SX2UC,Brooke,"December 30, 2003",Love This Phone,"This is a great, reliable phone. I also purcha...",Positive
3,3,B0000SX2UC,amy m. teague,"March 18, 2004","Love the Phone, BUT...!","I love the phone and all, because I really did...",Positive
4,4,B0000SX2UC,tristazbimmer,"August 28, 2005","Great phone service and options, lousy case!",The phone has been great for every purpose it ...,Positive
...,...,...,...,...,...,...,...
489,495,B002WTC1NG,Ryan M.,"December 16, 2014",Four Stars,Great flip phone!,Positive
490,496,B002WTC1NG,Ronald G. Dolter,"April 9, 2015",Five Stars,It arrived within the time quoted and is funct...,Neutral
491,497,B002WTC1NG,Thomas B. Reece,"May 14, 2015",Three Stars,"Phone worked ok, however the charger NEVER wor...",Positive
492,498,B002WTC1NG,sasha ford,"March 26, 2015",Five Stars,Very pleased with the speedy arrival and quali...,Positive


In [33]:
#slicing roberta results
new_df = pd.DataFrame(results_df)

final_result = new_df[['roberta_neg','roberta_neu','roberta_pos']]
final_result.head()

,roberta_neg,roberta_neu,roberta_pos
0,0.387844,0.342091,0.270064
1,0.459493,0.425768,0.114739
2,0.134225,0.299039,0.566736
3,0.009765,0.021803,0.968432
4,0.236579,0.328686,0.434734


In [34]:
#max value id

MaxValues = final_result.idxmax(axis=1)
final_df = pd.DataFrame(MaxValues)
final_df.head()

,0
0,roberta_neg
1,roberta_neg
2,roberta_pos
3,roberta_pos
4,roberta_pos


In [35]:
#replace
final_df.replace(['roberta_neg', 'roberta_pos','roberta_neu'], ['Negative', 'Positive','Neutral'], inplace=True)

final_df.head()

,0
0,Negative
1,Negative
2,Positive
3,Positive
4,Positive


In [36]:
ddresults_df = pd.DataFrame(results_df)
ddresults_df["Sentiment_RoBERTa"] = final_df
ddresults_df.head()

,Id,vader_neg,vader_neu,vader_pos,vader_compound,roberta_neg,roberta_neu,roberta_pos,asin,name,rating,date,verified,title,body,helpfulVotes,Sentiment_RoBERTa
0,0,0.080,0.816,0.105,0.8629,0.387844,0.342091,0.270064,B0000SX2UC,Janet,3,"October 11, 2005",False,"Def not best, but not worst",I had the Samsung A600 for awhile which is abs...,1.0,Negative
1,1,0.020,0.876,0.104,0.8860,0.459493,0.425768,0.114739,B0000SX2UC,Luke Wyatt,1,"January 7, 2004",False,Text Messaging Doesn't Work,Due to a software issue between Nokia and Spri...,17.0,Negative
2,2,0.051,0.846,0.103,0.7992,0.134225,0.299039,0.566736,B0000SX2UC,Brooke,5,"December 30, 2003",False,Love This Phone,"This is a great, reliable phone. I also purcha...",5.0,Positive
3,3,0.000,0.844,0.156,0.9592,0.009765,0.021803,0.968432,B0000SX2UC,amy m. teague,3,"March 18, 2004",False,"Love the Phone, BUT...!","I love the phone and all, because I really did...",1.0,Positive
4,4,0.066,0.814,0.121,0.7745,0.236579,0.328686,0.434734,B0000SX2UC,tristazbimmer,4,"August 28, 2005",False,"Great phone service and options, lousy case!",The phone has been great for every purpose it ...,1.0,Positive


In [37]:
#now remove the unnecessary columns

roberta_df = pd.DataFrame(ddresults_df)

def remove_unnecessary_columns(roberta_df, columns_to_remove):
    for column in columns_to_remove:
        
        if column in roberta_df.columns:
            roberta_df.drop(column, axis=1, inplace=True)
        else:
            break

    return roberta_df

In [38]:
roberta_df =roberta_df[['Id','vader_neg','vader_pos','vader_neu','vader_compound','roberta_neg','roberta_neu','roberta_pos','asin','name','date','verified','title','body','helpfulVotes','Sentiment_RoBERTa']]

columns_to_remove = ['roberta_neg','vader_neg','vader_pos','vader_neu','vader_compound','roberta_neu','roberta_pos','verified','helpfulVotes']

roberta_df = remove_unnecessary_columns(roberta_df, columns_to_remove)
roberta_df.head(500)

,Id,asin,name,date,title,body,Sentiment_RoBERTa
0,0,B0000SX2UC,Janet,"October 11, 2005","Def not best, but not worst",I had the Samsung A600 for awhile which is abs...,Negative
1,1,B0000SX2UC,Luke Wyatt,"January 7, 2004",Text Messaging Doesn't Work,Due to a software issue between Nokia and Spri...,Negative
2,2,B0000SX2UC,Brooke,"December 30, 2003",Love This Phone,"This is a great, reliable phone. I also purcha...",Positive
3,3,B0000SX2UC,amy m. teague,"March 18, 2004","Love the Phone, BUT...!","I love the phone and all, because I really did...",Positive
4,4,B0000SX2UC,tristazbimmer,"August 28, 2005","Great phone service and options, lousy case!",The phone has been great for every purpose it ...,Positive
...,...,...,...,...,...,...,...
489,495,B002WTC1NG,Ryan M.,"December 16, 2014",Four Stars,Great flip phone!,Positive
490,496,B002WTC1NG,Ronald G. Dolter,"April 9, 2015",Five Stars,It arrived within the time quoted and is funct...,Positive
491,497,B002WTC1NG,Thomas B. Reece,"May 14, 2015",Three Stars,"Phone worked ok, however the charger NEVER wor...",Negative
492,498,B002WTC1NG,sasha ford,"March 26, 2015",Five Stars,Very pleased with the speedy arrival and quali...,Positive


In [39]:
compare_df = pd.DataFrame(vader_df)
compare_df['Sentiment_RoBERTa'] = final_df
compare_df.head(500)

,Id,asin,name,date,title,body,Sentiment_vader,Sentiment_RoBERTa
0,0,B0000SX2UC,Janet,"October 11, 2005","Def not best, but not worst",I had the Samsung A600 for awhile which is abs...,Positive,Negative
1,1,B0000SX2UC,Luke Wyatt,"January 7, 2004",Text Messaging Doesn't Work,Due to a software issue between Nokia and Spri...,Positive,Negative
2,2,B0000SX2UC,Brooke,"December 30, 2003",Love This Phone,"This is a great, reliable phone. I also purcha...",Positive,Positive
3,3,B0000SX2UC,amy m. teague,"March 18, 2004","Love the Phone, BUT...!","I love the phone and all, because I really did...",Positive,Positive
4,4,B0000SX2UC,tristazbimmer,"August 28, 2005","Great phone service and options, lousy case!",The phone has been great for every purpose it ...,Positive,Positive
...,...,...,...,...,...,...,...,...
489,495,B002WTC1NG,Ryan M.,"December 16, 2014",Four Stars,Great flip phone!,Positive,Positive
490,496,B002WTC1NG,Ronald G. Dolter,"April 9, 2015",Five Stars,It arrived within the time quoted and is funct...,Neutral,Positive
491,497,B002WTC1NG,Thomas B. Reece,"May 14, 2015",Three Stars,"Phone worked ok, however the charger NEVER wor...",Positive,Negative
492,498,B002WTC1NG,sasha ford,"March 26, 2015",Five Stars,Very pleased with the speedy arrival and quali...,Positive,Positive
